In [1]:
import pandas as pd
import numpy as np

# XGBoostRandomForest


## Préparation des données V1 - 2023/11/15


Stratégie :  
On remplace les heures par des floats, les identifiants de station par des entiers et on supprime la colonne 'date'.  
(La suppression de la colonne 'way' a été oubliée)
On a fait une division : 90% du dataset pour le train, 10% pour le test


- Import des données


In [2]:
path = "./../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate

- Mise en forme des données


In [3]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data = x_data.drop("way", axis=1)  # on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(
    lambda x: int(x[:2]) if isinstance(x, str) else np.nan
)  # on transforme les strings des heures en float
# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Création d'un dataset de test et de validation


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x_data.values, y_data.values, train_size=0.8
)

## XGBoostREGRESSOR - 2023/11/13 - Préparation des données V1


On utilise le regresseur et pas le classifieur car on ne cherche pas la classe de p0q0 mais sa valeur précise.


- Import de XGBRegresseur


In [5]:
from xgboost import XGBRFRegressor

- Test sans jouer sur les paramètres


In [6]:
XGBRF = XGBRFRegressor()
XGBRF.fit(X_train, y_train)
XGBRF.score(X_test, y_test)

0.9534488145772185

Le score est moins bien que les autres méthodes on va tenter de l'optimiser.


- Choix de learning_rate et n_estimators


In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from time import time

start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

n_estimators = np.arange(600, 650, 10)
learning_rate = np.linspace(0.8, 1.1, 10)

tuned_parameters = {
    "n_estimators": n_estimators,
    "learning_rate": learning_rate,
}

XGBRF_params = GridSearchCV(
    XGBRFRegressor(), tuned_parameters, cv=my_kfold, n_jobs=-1
)

XGBRF_params.fit(X_train, y_train)
print(XGBRF_params.score(X_test, y_test), XGBRF_params.best_params_)
print(str(time() - start) + " sec")

0.9535046268206377 {'learning_rate': 1.0, 'n_estimators': 610}
267.37779450416565 sec


- choix de max_depth et min_child_weight


In [ ]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

max_depth = [24]
min_child_weight = [2]

tuned_parameters = {
    "max_depth": max_depth,
    "min_child_weight": min_child_weight,
}

XGBRF_params = GridSearchCV(
    XGBRFRegressor(n_estimators=650, learning_rate=1),
    tuned_parameters,
    cv=my_kfold,
    n_jobs=-1,
)

XGBRF_params.fit(X_train, y_train)
print(XGBRF_params.score(X_test, y_test), XGBRF_params.best_params_)
print(str(time() - start) + " sec")

0.9881470166124114 {'max_depth': 24, 'min_child_weight': 2}
192.11008214950562 sec


Premier test avec

- max_depth = [2, 4, 10, 20]
- min_child_weight = [5, 10, 15]

On obtient :

0.987933888068767 {'max_depth': 20, 'min_child_weight': 5}
371.24381494522095 sec


Deuxième test avec


Troisième test avec


On choisit max_depth = 7 et min_child_weight = 8


- choix de reg_alpha et reg_lambda


In [ ]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

reg_alpha = [0.001, 0.005, 0.01]
reg_lambda = [0.001, 0.005, 0.01]

tuned_parameters = {"reg_alpha": reg_alpha, "reg_lambda": reg_lambda}

XGBRF_params = GridSearchCV(
    XGBRFRegressor(
        n_estimators=650, learning_rate=1, max_depth=25, min_child_weight=4
    ),
    tuned_parameters,
    cv=my_kfold,
    n_jobs=-1,
)

XGBRF_params.fit(X_train, y_train)
print(XGBRF_params.score(X_test, y_test), XGBRF_params.best_params_)
print(str(time() - start) + " sec")

0.9877853842432953 {'reg_alpha': 0.001, 'reg_lambda': 0.001}
82.71706700325012 sec


Premier test avec

- reg_alpha = [0.04, 0.08, 0.1, 0.14, 0.15]
- reg_lambda = [0.05, 0.07, 0.10, 0.13, 0.15]

On obtient :

0.983133818285855 {'reg_alpha': 0.04, 'reg_lambda': 0.05}
278.0309364795685 sec


Deuxième test avec

- reg_alpha = [0.01, 0.02, 0.04]
- reg_lambda = [0.01, 0.03, 0.05]

On obtient:
0.9865282567140496 {'reg_alpha': 0.01, 'reg_lambda': 0.01}
142.79405546188354 sec


Troisième test avec

- reg_alpha = [0.001, 0.005, 0.01]
- reg_lambda = [0.001, 0.005, 0.01]

  0.9877853842432953 {'reg_alpha': 0.001, 'reg_lambda': 0.001}
  272.71706700325012 sec


On réduit à nouveau pour voir si il est possible d'améliorer le score


Quatrième test avec

- reg_alpha = [0.0001, 0.0005, 0.001]
- reg_lambda = [0.0001, 0.0005, 0.001]

  0.9878969094548209 {'reg_alpha': 0.0001, 'reg_lambda': 0.0001}
  270.20020604133606 sec


Ca ne sert à rien de continuer l'amélioration est mineure


##### Test finaux


In [ ]:
XGBRF1 = XGBRFRegressor(
    n_estimators=650,
    learning_rate=1,
    max_depth=25,
    min_child_weight=3,
    reg_alpha=0.001,
    reg_lambda=0.001,
)
XGBRF1.fit(X_train, y_train)
XGBRF1.score(X_test, y_test)

0.98797022454064

In [ ]:
XGBRF2 = XGBRFRegressor(
    n_estimators=750,
    learning_rate=1,
    max_depth=25,
    min_child_weight=3,
    reg_alpha=0.001,
    reg_lambda=0.001,
)
XGBRF2.fit(X_train, y_train)
XGBRF2.score(X_test, y_test)

0.9879740055922637